In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from estudo import Estudo

dic = Estudo(
    ano = 2024,
    id_modulo = None,
    analito = None
).run()

In [2]:
erros = dic.get('erros')
len(erros)

370

In [3]:
modulos = sorted([int(i) for i in dic.keys() if i != 'erros'])
len(modulos)

291

In [5]:
dados = []
for mod in modulos:
    dados.extend(dic.get(mod)) 

In [52]:
cont = 0
clusters = 0
for data in dados:
    if len(i:=(data.query('identico=="Sim"'))) > 0:
        cont += len(i.cluster.unique())
    qtd_clusters = len(data.cluster.unique())
    clusters += qtd_clusters
cont, clusters

(284, 1570)

In [51]:
cont = 0
clusters = 0
for data in dados:
    if len(i:=(data.query('identico=="Não"'))) > 0:
        cont += len(i.cluster.unique())
    qtd_clusters = len(data.cluster.unique())
    clusters += qtd_clusters
cont, clusters

(1311, 1570)

In [ ]:
cont = 0
for data in dados:
    cont += (len(data.groupby('cluster', as_index=False).identico.agg(lambda x: len(set(x))).query('identico>1')))
    
cont


25

In [64]:
284+1311

1595

In [116]:
teste = pd.concat(dados)

### Número de grupos suspeitos por módulo (Top 10)


In [131]:
teste['grupos'] = teste['analito'] + ' -- ' + teste['cluster'].astype(str)
teste2 = teste.groupby(['id_modulo'], as_index=False).grupos.agg(lambda x: len(set(x))).sort_values('grupos', ascending=False).head(10)
teste2['id_modulo'] = teste2['id_modulo'].astype(str)

import plotly.express as px

px.bar(
    data_frame=teste2,
    x='grupos',
    y='id_modulo',
    title='Top 10 módulos com mais grupos suspeitos em colusão',
    text_auto=True
)

### 

In [6]:
from utils import Data

dados = Data().get_data(2024, None, None)
len(dados), len(dados.part.unique())

(1426101, 3690)

In [7]:
import hashlib
from estudo import Estudo
import pandas as pd
pd.set_option('display.max_columns', None)

dic = Estudo(2024, [58], None).run()

df = dic.get(58)[5].query('identico=="Não"')

itens = [col for col in df.columns if 'A24' in col]


# Função para gerar hash (exemplo com SHA-256)
def codificar_hash(numero):
    return hashlib.sha256(str(numero).encode()).hexdigest()[:8]  # 8 primeiros caracteres


df['part'] = df['part'].apply(codificar_hash)
df['grupo'] = df['grupo'].apply(lambda x: codificar_hash(x) if x!=None else '')

df = df[['id_modulo','analito','part','regiao','grupo','sistema','eps_calculado','cluster','identico']+itens]

df

C:\Users\brunocavalcanti\AppData\Local\Temp\ipykernel_11688\4120556190.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['part'] = df['part'].apply(codificar_hash)
C:\Users\brunocavalcanti\AppData\Local\Temp\ipykernel_11688\4120556190.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['grupo'] = df['grupo'].apply(lambda x: codificar_hash(x) if x!=None else '')


,id_modulo,analito,part,regiao,grupo,sistema,eps_calculado,cluster,identico,A24M10I01,A24M10I02,A24M10I03,A24M1I01,A24M1I02,A24M1I03,A24M4I01,A24M4I02,A24M4I03,A24M7I01,A24M7I02,A24M7I03
17,58,CPK,df2760a9,SP,a258b30f,1,0.043707,8,Não,126.0,109.0,110.0,231.0,201.0,125.0,127.0,231.0,270.0,209.0,138.0,114.0
18,58,CPK,37e0529a,SP,a258b30f,1,0.043707,8,Não,126.0,109.0,110.0,230.0,199.0,125.0,127.0,230.0,269.0,208.0,137.0,113.0
